In [1]:
# Import necessary modules
import numpy as np
import pandas as pd

#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

C:\Users\narayanan.p\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Load the data frams from csv file
df_other_specs = pd.read_csv("other_specs.csv")
df_fund_specs = pd.read_csv("fund_specs.csv")
df_fund_ratios = pd.read_csv("fund_ratios.csv")
df_submission_file = pd.read_csv("sample_submission.csv")

df_return_3Y = pd.read_csv("return_3year.csv")
#df_return_5Y = pd.read_csv("return_5year.csv")
#df_return_10Y = pd.read_csv("return_10year.csv")

## Others to be considered later are bond allocations for blend & value funds


In [4]:
df_linkage = df_fund_ratios.filter(['fund_id', 'tag'],axis=1)

In [5]:
## Create features that may be useful in each df
#1. Fund_Specs: None to be created
df_fund_specs.drop(columns=['currency','total_assets', 'inception_date'], inplace=True)

#2. Other_Specs : Create one feature for each (fund value / category value)
df_other_specs['2018_return_ratio'] = df_other_specs['2018_return_fund']/df_other_specs['2018_return_category']
df_other_specs['2017_return_ratio'] = df_other_specs['2017_return_fund']/df_other_specs['2017_category_return']
df_other_specs['2016_return_ratio'] = df_other_specs['2016_return_fund']/df_other_specs['2016_return_category']
df_other_specs['2016_return_ratio'] = df_other_specs['2016_return_ratio'].replace((np.inf, -np.inf), (0, 0))
df_other_specs['2015_return_ratio'] = df_other_specs['2015_return_fund']/df_other_specs['category_return_2015']
df_other_specs['2014_return_ratio'] = df_other_specs['2014_return_fund']/df_other_specs['2014_category_return']
df_other_specs['2013_return_ratio'] = df_other_specs['2013_return_fund']/df_other_specs['2013_category_return']
df_other_specs['2012_return_ratio'] = df_other_specs['2012_fund_return']/df_other_specs['2012_return_category']
df_other_specs['2011_return_ratio'] = df_other_specs['2011_return_fund']/df_other_specs['2011_return_category']
df_other_specs['2010_return_ratio'] = df_other_specs['2010_return_fund']/df_other_specs['2010_return_category']

df_other_specs['1Y_return_ratio'] = df_other_specs['1_year_return_fund']/df_other_specs['category_return_1year']
df_other_specs['1Y_return_ratio']=  df_other_specs['1Y_return_ratio'].replace((np.inf, -np.inf), (0, 0))

#df_other_specs['category_return_1year']
#df_other_specs['category_return_1year'] =df_other_specs['category_return_1year'].where((df_other_specs['category_return_1year']==0)|(df_other_specs['category_return_1year'].isnull()),0.00001)
#df_other_specs['1Y_return_ratio'] = df_other_specs['1_year_return_fund']/df_other_specs['category_return_1year']
#df_other_specs['1Y_return_ratio'] = (df_other_specs['1_year_return_fund']/df_other_specs['category_return_1year']).where((df_other_specs['category_return_1year'].notnull()) | (df_other_specs['category_return_1year']==0) ,0)

df_other_specs['3M_return_ratio'] = df_other_specs['fund_return_3months']/df_other_specs['3_months_return_category']
df_other_specs['3M_return_ratio'] = df_other_specs['3M_return_ratio'].replace((np.inf, -np.inf), (0, 0))
df_other_specs['1M_return_ratio'] = df_other_specs['1_month_fund_return']/df_other_specs['category_return_1month']
df_other_specs['1M_return_ratio'] = df_other_specs['1M_return_ratio'].replace((np.inf, -np.inf), (0, 0))
df_other_specs['ytd_return_ratio'] = df_other_specs['ytd_return_fund']/df_other_specs['ytd_return_category']
df_other_specs['ytd_return_ratio'] = df_other_specs['ytd_return_ratio'].replace((np.inf, -np.inf), (0, 0))

df_other_specs['years_up_down'] = df_other_specs['years_up']-df_other_specs['years_down']
df_other_specs['cash_to_stock_ratio'] = (df_other_specs['cash_percent_of_portfolio']+df_other_specs['bond_percentage_of_porfolio'])/df_other_specs['stock_percent_of_portfolio']
df_other_specs['cash_to_stock_ratio']=  df_other_specs['cash_to_stock_ratio'].replace((np.inf, -np.inf), (0, 0))
df_other_specs.drop(columns=[
    '2018_return_fund','2018_return_category',
    '2017_return_fund','2017_category_return',
    '2016_return_fund','2016_return_category',
    '2015_return_fund','category_return_2015',
    '2014_return_fund','2014_category_return',
    '2013_return_fund','2013_category_return',
    '2012_fund_return','2012_return_category',
    '2011_return_fund','2011_return_category',
    '2010_return_fund','2010_return_category',
    '1_year_return_fund','category_return_1year',
    'fund_return_3months','3_months_return_category',
    '1_month_fund_return','category_return_1month',
    'ytd_return_fund','ytd_return_category',
    'cash_percent_of_portfolio', 'stock_percent_of_portfolio','bond_percentage_of_porfolio',
    'years_up','years_down','greatstone_rating'], inplace=True)

## DF3 : Fund Ratios : 
df_fund_ratios.drop(columns=['mmc','fund_id','pb_ratio','ps_ratio','pc_ratio','pe_ratio'],inplace=True)


#DF4 : 3 year return : Retain = fund_return_3years, 3_years_return_category

df_return_3Y['3yrs_treynor_ratio_fund'] = pd.to_numeric(df_return_3Y['3yrs_treynor_ratio_fund'], downcast="float",errors='coerce')

df_return_3Y['3y_treynor_ratio'] = df_return_3Y['3yrs_treynor_ratio_fund']/df_return_3Y['3yrs_treynor_ratio_category']
df_return_3Y['3y_alpha_ratio'] = df_return_3Y['3_years_alpha_fund']/df_return_3Y['3_years_alpha_category']
df_return_3Y['3y_sharpe_ratio'] = df_return_3Y['3yrs_sharpe_ratio_fund']/df_return_3Y['3yrs_sharpe_ratio_category']
df_return_3Y['3y_rma_ratio'] = df_return_3Y['3_years_return_mean_annual_fund']/df_return_3Y['3_years_return_mean_annual_category']
df_return_3Y['3y_beta_ratio'] = df_return_3Y['fund_beta_3years']/df_return_3Y['category_beta_3years']
df_return_3Y['3y_r2_ratio'] = df_return_3Y['3years_fund_r_squared']/df_return_3Y['3years_category_r_squared']
df_return_3Y['3y_std_ratio'] = df_return_3Y['3years_fund_std']/df_return_3Y['3years_category_std']
df_return_3Y['3y_return_ratio'] = df_return_3Y['fund_return_3years']/df_return_3Y['3_years_return_category']

df_return_3Y.drop(columns=[
    '3yrs_treynor_ratio_fund','3yrs_treynor_ratio_category',
    '3_years_alpha_fund','3_years_alpha_category',
    '3yrs_sharpe_ratio_fund','3yrs_sharpe_ratio_category',
    '3_years_return_mean_annual_fund','3_years_return_mean_annual_category',
    'fund_beta_3years','category_beta_3years',
    '3years_fund_r_squared','3years_category_r_squared',
    '3years_fund_std','3years_category_std',
    'fund_return_3years','3_years_return_category'],inplace=True)

#### DF 5Y & 10 year can be added later
#DF5 : 5 year return : Retain = 5_years_return_fund, 5_years_return_category
#df_return_5Y = df_return_5Y.filter(['tag','5_years_return_fund', '5_years_return_category'],axis=1)


#DF6 : 10 year return : Retain = 10_years_return_fund, 10_years_return_category
#df_return_10Y = df_return_10Y.filter(['fund_id','10_years_return_fund', '10_years_return_category'],axis=1)


In [6]:
## Create master data
# DF1 : Linkage. Drop no columns
# DF2 : Fund_Specs 
df_dataset = pd.merge(df_linkage,df_fund_specs,on='tag')

#DF3 : other_specs 
df_dataset = pd.merge(df_dataset,df_other_specs,on='tag')

# DF4 : Fund ratios : 
df_dataset = pd.merge(df_dataset,df_fund_ratios,on='tag')

#DF5 : 3 year return :
df_return_3Y = df_return_3Y.filter(['tag','fund_return_3years', '3_years_return_category'],axis=1)
df_dataset = pd.merge(df_dataset,df_return_3Y,on='tag')

#print(df_dataset.columns)
#print(df_dataset.shape)

In [7]:
# Columsn as object are converted to float
df_dataset['pc_ratio'] = pd.to_numeric(df_dataset['pc_ratio'], downcast="float",errors='coerce')
df_dataset['pb_ratio'] = pd.to_numeric(df_dataset['pb_ratio'], downcast="float",errors='coerce')
df_dataset['ps_ratio'] = pd.to_numeric(df_dataset['ps_ratio'], downcast="float",errors='coerce')
df_dataset['pe_ratio'] = pd.to_numeric(df_dataset['pe_ratio'], downcast="float",errors='coerce')
df_dataset['mmc'] = pd.to_numeric(df_dataset['mmc'], downcast="float",errors='coerce')

In [8]:
# Data cleaning

In [9]:
#Impute blanks in investment_class with 'Unkowwn'
df_dataset['investment_class'] = df_dataset['investment_class'].fillna('Unknown')

In [10]:
#Impute blanks in fund_size with 'Unkowwn'
#print(df_dataset['fund_size'].isna().sum())
df_dataset['fund_size'] = df_dataset['fund_size'].fillna('Unknown')
#print(df_dataset['fund_size'].isna().sum())

In [11]:
#print(df_dataset['yield'].isna().sum())
# Impute missing yield with zero [may need to change later]
df_dataset['yield'] = df_dataset['yield'].fillna(0)
#print(df_dataset['yield'].isna().sum())

In [12]:
exclude = ['fund_id','tag','investment_class','greatstone_rating','fund_size']
for i in df_dataset.columns:
    if i not in exclude :
        df_dataset[i] = round(pd.to_numeric(df_dataset[i]),3)
        df_dataset[i] = df_dataset[i].fillna(0)

In [13]:
df_dataset.to_csv("dataset_full21.csv")

In [14]:
df_Value = df_dataset[df_dataset['investment_class']=='Value']
df_Blend = df_dataset[df_dataset['investment_class']=='Blend']
df_Growth = df_dataset[df_dataset['investment_class']=='Growth']
df_Unknown = df_dataset[df_dataset['investment_class']=='Unknown']

In [16]:
mms = MinMaxScaler()

data_subsets = [df_Value,df_Blend,df_Growth,df_Unknown]
for df in data_subsets:
    exclude = ['fund_id','tag','investment_class','yield','greatstone_rating','fund_size']
    for i in df.columns:
        if i not in exclude :            
            df[i] = pd.to_numeric(df[i])
            df[i] = df[i].fillna(df[i].mean())
    
    # Following code could be on the full dataset, however doiing it here as this code is for encoding. 
    df.fund_size[df.fund_size == 'Unknown']=0
    df.fund_size[df.fund_size == 'Small']=1
    df.fund_size[df.fund_size == 'Medium']=2
    df.fund_size[df.fund_size == 'Large']=3
    
    #Min-max scale all data columns

    df[['yield', 
           'fund_size', 'return_ytd', 'pc_ratio', 'pb_ratio', 'pe_ratio',
           'portfolio_convertable', 'portfolio_others', 'mmc', 'ps_ratio',
           'fund_return_3years', 'category_ratio_net_annual_expense',
           'portfolio_preferred', '2018_return_ratio', '2017_return_ratio',
           '2016_return_ratio', '2015_return_ratio', '2014_return_ratio',
           '2013_return_ratio', '2012_return_ratio', '2011_return_ratio',
           '2010_return_ratio', '1Y_return_ratio', '3M_return_ratio',
           '1M_return_ratio', 'ytd_return_ratio', 'years_up_down',
           'cash_to_stock_ratio', 'fund_ratio_net_annual_expense']] = mms.fit_transform(df[['yield', 
           'fund_size', 'return_ytd', 'pc_ratio', 'pb_ratio', 'pe_ratio',
           'portfolio_convertable', 'portfolio_others', 'mmc', 'ps_ratio',
           'fund_return_3years', 'category_ratio_net_annual_expense',
           'portfolio_preferred', '2018_return_ratio', '2017_return_ratio',
           '2016_return_ratio', '2015_return_ratio', '2014_return_ratio',
           '2013_return_ratio', '2012_return_ratio', '2011_return_ratio',
           '2010_return_ratio', '1Y_return_ratio', '3M_return_ratio',
           '1M_return_ratio', 'ytd_return_ratio', 'years_up_down',
           'cash_to_stock_ratio', 'fund_ratio_net_annual_expense']])


C:\Users\narayanan.p\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\narayanan.p\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\narayanan.p\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

C:\Users\narayanan.p\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
df_corr = df_dataset.corr()
df_corr.to_csv("correlation1.csv")

In [18]:
## Fulll set of code for VALUE
df_Value_train = df_Value[df_Value.greatstone_rating.notnull()]
df_Value_test = df_Value[df_Value.greatstone_rating.isnull()]


## 'New Code'
df_linkage = df_dataset.filter(['fund_id', 'tag'],axis=1)
df_linkage_Value = df_Value.filter(['fund_id', 'tag'],axis=1)
df_Value_train_keys = df_Value_train.filter(['tag'],axis=1)
df_Value_test_keys = df_Value_test.filter(['tag'],axis=1)

keys_train = list(df_Value_train_keys.columns.values)
i1 = df_linkage_Value.set_index(keys_train).index
i2 = df_Value_train_keys.set_index(keys_train).index
df_Value_linkage_train = df_linkage_Value[i1.isin(i2)]
df_Value_linkage_test = df_linkage_Value[~i1.isin(i2)]
df_Value_train_X = df_Value_train.drop(labels='greatstone_rating',axis=1)
df_Value_train_y = df_Value_train['greatstone_rating']

# Drop funid & tag columns in both test & train 
df_Value_train_X.drop(columns=['fund_id','tag'],inplace=True)
df_Value_test.drop(columns=['fund_id','tag','greatstone_rating'],inplace=True)

# OneHot encode investment class in both train_X & test
inv_class = pd.get_dummies(df_Value_train_X['investment_class'],drop_first=True)
df_Value_train_X.drop(['investment_class'],axis=1,inplace=True)
df_Value_train_X = pd.concat([df_Value_train_X,inv_class],axis=1)

inv_class = pd.get_dummies(df_Value_test['investment_class'],drop_first=True)
df_Value_test.drop(['investment_class'],axis=1,inplace=True)
df_Value_test = pd.concat([df_Value_test,inv_class],axis=1)

(5250, 33)
(1301, 33)
(25000, 2)
(6551, 2)
(5250, 1)
(1301, 1)
['tag']
(5250, 2)
(1301, 2)
(5250, 30)
(1301, 30)
(5250, 29)
(1301, 29)


C:\Users\narayanan.p\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
## Fulll set of code for Blend
df_Blend_train = df_Blend[df_Blend.greatstone_rating.notnull()]
df_Blend_test = df_Blend[df_Blend.greatstone_rating.isnull()]
print(df_Blend_train.shape)
print(df_Blend_test.shape)

## 'New Code'
df_linkage = df_dataset.filter(['fund_id', 'tag'],axis=1)

df_linkage_Blend = df_Blend.filter(['fund_id', 'tag'],axis=1)

df_Blend_train_keys = df_Blend_train.filter(['tag'],axis=1)
df_Blend_test_keys = df_Blend_test.filter(['tag'],axis=1)

keys_train = list(df_Blend_train_keys.columns.values)
i1 = df_linkage_Blend.set_index(keys_train).index
i2 = df_Blend_train_keys.set_index(keys_train).index
df_Blend_linkage_train = df_linkage_Blend[i1.isin(i2)]
df_Blend_linkage_test = df_linkage_Blend[~i1.isin(i2)]

df_Blend_train_X = df_Blend_train.drop(labels='greatstone_rating',axis=1)
df_Blend_train_y = df_Blend_train['greatstone_rating']

# Drop funid & tag columns in both test & train 
df_Blend_train_X.drop(columns=['fund_id','tag'],inplace=True)
df_Blend_test.drop(columns=['fund_id','tag','greatstone_rating'],inplace=True)

# OneHot encode investment class in both train_X & test
inv_class = pd.get_dummies(df_Blend_train_X['investment_class'],drop_first=True)
df_Blend_train_X.drop(['investment_class'],axis=1,inplace=True)
df_Blend_train_X = pd.concat([df_Blend_train_X,inv_class],axis=1)

inv_class = pd.get_dummies(df_Blend_test['investment_class'],drop_first=True)
df_Blend_test.drop(['investment_class'],axis=1,inplace=True)
df_Blend_test = pd.concat([df_Blend_test,inv_class],axis=1)

(8217, 33)
(2081, 33)


In [21]:
## Fulll set of code for Growth
df_Growth_train = df_Growth[df_Growth.greatstone_rating.notnull()]
df_Growth_test = df_Growth[df_Growth.greatstone_rating.isnull()]
print(df_Growth_train.shape)
print(df_Growth_test.shape)

## 'New Code'
df_linkage = df_dataset.filter(['fund_id', 'tag'],axis=1)

df_linkage_Growth = df_Growth.filter(['fund_id', 'tag'],axis=1)

df_Growth_train_keys = df_Growth_train.filter(['tag'],axis=1)
df_Growth_test_keys = df_Growth_test.filter(['tag'],axis=1)

keys_train = list(df_Growth_train_keys.columns.values)
i1 = df_linkage_Growth.set_index(keys_train).index
i2 = df_Growth_train_keys.set_index(keys_train).index
df_Growth_linkage_train = df_linkage_Growth[i1.isin(i2)]
df_Growth_linkage_test = df_linkage_Growth[~i1.isin(i2)]

df_Growth_train_X = df_Growth_train.drop(labels='greatstone_rating',axis=1)
df_Growth_train_y = df_Growth_train['greatstone_rating']

# Drop funid & tag columns in both test & train 
df_Growth_train_X.drop(columns=['fund_id','tag'],inplace=True)
df_Growth_test.drop(columns=['fund_id','tag','greatstone_rating'],inplace=True)

# OneHot encode investment class in both train_X & test
inv_class = pd.get_dummies(df_Growth_train_X['investment_class'],drop_first=True)
df_Growth_train_X.drop(['investment_class'],axis=1,inplace=True)
df_Growth_train_X = pd.concat([df_Growth_train_X,inv_class],axis=1)

inv_class = pd.get_dummies(df_Growth_test['investment_class'],drop_first=True)
df_Growth_test.drop(['investment_class'],axis=1,inplace=True)
df_Growth_test = pd.concat([df_Growth_test,inv_class],axis=1)

(5328, 33)
(1343, 33)


In [22]:
## Fulll set of code for Unknown
df_Unknown_train = df_Unknown[df_Unknown.greatstone_rating.notnull()]
df_Unknown_test = df_Unknown[df_Unknown.greatstone_rating.isnull()]
print(df_Unknown_train.shape)
print(df_Unknown_test.shape)

## 'New Code'
df_linkage = df_dataset.filter(['fund_id', 'tag'],axis=1)

df_linkage_Unknown = df_Unknown.filter(['fund_id', 'tag'],axis=1)

df_Unknown_train_keys = df_Unknown_train.filter(['tag'],axis=1)
df_Unknown_test_keys = df_Unknown_test.filter(['tag'],axis=1)

keys_train = list(df_Unknown_train_keys.columns.values)
i1 = df_linkage_Unknown.set_index(keys_train).index
i2 = df_Unknown_train_keys.set_index(keys_train).index
df_Unknown_linkage_train = df_linkage_Unknown[i1.isin(i2)]
df_Unknown_linkage_test = df_linkage_Unknown[~i1.isin(i2)]

df_Unknown_train_X = df_Unknown_train.drop(labels='greatstone_rating',axis=1)
df_Unknown_train_y = df_Unknown_train['greatstone_rating']

# Drop funid & tag columns in both test & train 
df_Unknown_train_X.drop(columns=['fund_id','tag'],inplace=True)
df_Unknown_test.drop(columns=['fund_id','tag','greatstone_rating'],inplace=True)

# OneHot encode investment class in both train_X & test
inv_class = pd.get_dummies(df_Unknown_train_X['investment_class'],drop_first=True)
df_Unknown_train_X.drop(['investment_class'],axis=1,inplace=True)
df_Unknown_train_X = pd.concat([df_Unknown_train_X,inv_class],axis=1)

inv_class = pd.get_dummies(df_Unknown_test['investment_class'],drop_first=True)
df_Unknown_test.drop(['investment_class'],axis=1,inplace=True)
df_Unknown_test = pd.concat([df_Unknown_test,inv_class],axis=1)

(1205, 33)
(275, 33)


In [23]:
## MODEL 3 : RAMDOM FOREST
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=25, bootstrap = True, max_features = 'sqrt')
rf_model.fit(df_Value_train_X,df_Value_train_y)
predictions = rf_model.predict(df_Value_test)

Value_predicted_ratings = pd.DataFrame(predictions)

df_Value_linkage_test.reset_index(drop=True, inplace=True)
Value_predicted_ratings.reset_index(drop=True, inplace=True)
df_Value_final_ratings = pd.concat([df_Value_linkage_test,Value_predicted_ratings],axis=1)
df_Value_final_ratings.drop(columns=['tag'],inplace=True)
print(df_Value_final_ratings.shape)
df_Value_final_ratings.to_csv("df_Value_final_ratings.csv")

(1301, 2)


In [24]:
rf_model1 = RandomForestClassifier(n_estimators=25, bootstrap = True, max_features = 'sqrt')
rf_model1.fit(df_Blend_train_X,df_Blend_train_y)
predictions1 = rf_model1.predict(df_Blend_test)

Blend_predicted_ratings = pd.DataFrame(predictions1)

df_Blend_linkage_test.reset_index(drop=True, inplace=True)
Blend_predicted_ratings.reset_index(drop=True, inplace=True)
df_Blend_final_ratings = pd.concat([df_Blend_linkage_test,Blend_predicted_ratings],axis=1)
df_Blend_final_ratings.drop(columns=['tag'],inplace=True)
print(df_Blend_final_ratings.shape)

(2081, 2)


In [25]:
rf_model2 = RandomForestClassifier(n_estimators=25, bootstrap = True, max_features = 'sqrt')
rf_model2.fit(df_Growth_train_X,df_Growth_train_y)
predictions2 = rf_model2.predict(df_Growth_test)

Growth_predicted_ratings = pd.DataFrame(predictions2)

df_Growth_linkage_test.reset_index(drop=True, inplace=True)
Growth_predicted_ratings.reset_index(drop=True, inplace=True)
df_Growth_final_ratings = pd.concat([df_Growth_linkage_test,Growth_predicted_ratings],axis=1)
df_Growth_final_ratings.drop(columns=['tag'],inplace=True)
print(df_Growth_final_ratings.shape)

(1343, 2)


In [27]:
rf_model3 = RandomForestClassifier(n_estimators=25, bootstrap = True, max_features = 'sqrt')
rf_model3.fit(df_Unknown_train_X,df_Unknown_train_y)
predictions3 = rf_model3.predict(df_Unknown_test)

Unknown_predicted_ratings = pd.DataFrame(predictions3)

df_Unknown_linkage_test.reset_index(drop=True, inplace=True)
Unknown_predicted_ratings.reset_index(drop=True, inplace=True)
df_Unknown_final_ratings = pd.concat([df_Unknown_linkage_test,Unknown_predicted_ratings],axis=1)
df_Unknown_final_ratings.drop(columns=['tag'],inplace=True)
print(df_Unknown_final_ratings.shape)

(275, 2)


In [28]:
df_Value_final_ratings.rename(columns={"0": "greatstone_rating"})
df_Growth_final_ratings.rename(columns={"0": "greatstone_rating"})
df_Blend_final_ratings.rename(columns={"0": "greatstone_rating"})
df_Unknown_final_ratings.rename(columns={"0": "greatstone_rating"})


,fund_id,0
0,468b76c0-a276-45b9-ad76-1a9d6c336ed6,2.0
1,cd9b1f78-e450-489f-b6e5-ece691d5c021,1.0
2,7723006e-720e-4c97-825f-1752cd112736,0.0
3,37c33326-b1fb-4208-a7df-08f7ead749da,3.0
4,d1052147-38d7-4981-8b8c-b3a0dcda8aed,2.0
...,...,...
270,668a58e7-5bee-41fd-9e23-963aa2f08f01,3.0
271,96c3122c-eea0-4d39-b4ed-c1647b03f834,0.0
272,80001be5-fde4-4f20-a8c3-ec295aed5960,0.0
273,4d555bb4-ada5-4a1c-8a01-0827477dd7e5,0.0


In [29]:
frames = [df_Value_final_ratings,df_Blend_final_ratings,df_Growth_final_ratings,df_Unknown_final_ratings]
final_result = pd.concat(frames)

In [30]:
final_result.shape

(5000, 2)

In [32]:
final_result.tail(20)

,fund_id,0
255,f0e1829f-0bcc-43e7-9759-c66efdca37d4,4.0
256,1e3ebfe5-644b-4a3b-92df-a94bdf8b22e0,3.0
257,1ec88986-c424-4d42-b3d1-f0a50d25e1b7,0.0
258,02ece6b9-00e2-4698-bbd5-a51793e3653e,4.0
259,e1c1618c-4e4b-4c22-b9ae-468a9abdf7a8,3.0
260,37392a99-d692-49bd-aece-b957cc6d16b3,2.0
261,9ecf9c73-fb54-4abf-b6c3-d10f8787ff4d,1.0
262,d2b9cae9-1e3e-4ce3-b614-eb37bde0d971,3.0
263,600d3d36-425c-47b2-803a-7fb2f3c100c6,2.0
264,1aa22f3d-1861-42a0-a850-e92f0f780f1e,4.0


In [33]:
# Reorder linkage_test to match the submission file
final_result = final_result.set_index('fund_id')
final_result = final_result.reindex(index=df_submission_file['fund_id'])
final_result = final_result.reset_index()
final_result.to_csv("Subission_Narayanan_SplitDF.csv")

In [ ]:
STOP HERE

In [ ]:
## MODEL 1 : Logistic regression
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
model = logmodel.fit(df_dataset_train_X,df_dataset_train_y)
predictions = logmodel.predict(df_dataset_test)


In [ ]:
## MODEL 2 : SVM/SVC
from sklearn.svm import SVC

# Building a Support Vector Machine on train data
svc_model = SVC(C= 0.1, kernel='linear', gamma= 1)
svc_model.fit(df_dataset_train_X,df_dataset_train_y)
predictions = svc_model.predict(df_dataset_test)

In [ ]:
df_dataset_train_X.dtypes

In [ ]:
## MODEL 3 : RAMDOM FOREST
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
rf_model.fit(df_dataset_train_X,df_dataset_train_y)
predictions = rf_model.predict(df_dataset_test)


In [ ]:
## USe PCA
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df_dataset_train_X)
X_pca = pca.transform(df_dataset_train_X)
X_pca.shape



In [ ]:
plt.figure(figsize=(10,7))

plt.plot(np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)

plt.xlabel('Number of components')
plt.ylabel('Total explained variance')

plt.xlim(0, 18)
plt.yticks(np.arange(0.9, 1.0, 0.05))

plt.axvline(2, c='b')

plt.axhline(0.95, c='r')
plt.axhline(0.96, c='r')
plt.axhline(0.97, c='r')
plt.axhline(0.98, c='r')
plt.axhline(0.99, c='r')
plt.show()

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
pca = PCA()
pca.fit(df_dataset_train_X)

In [ ]:
## MODEL 4 : RAMDOM FOREST with PCA
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
rf_model.fit(pca.transform(df_dataset_train_X), df_dataset_train_y)
predictions = rf_model.predict(pca.transform(df_dataset_test))


In [ ]:
# MODEL 5 :USE >LINEAR REGRESSION (MAY BE WASTE)
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()
regression_model.fit(df_dataset_train_X, df_dataset_train_y)
predictions = regression_model.predict(df_dataset_test)


In [ ]:
predictions

In [ ]:
predicted_ratings = pd.DataFrame(predictions)

In [ ]:
predicted_ratings = predicted_ratings.round(0)

In [ ]:
df_final_ratings = pd.concat([df_linkage_test,predicted_ratings],axis=1)
df_final_ratings.drop(columns=['tag'],inplace=True)
df_final_ratings.to_csv("Submission_Narayanan.csv")

In [ ]:
predicted_ratings